In [572]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [573]:
df1=pd.read_excel("C:/Users/L7927301/Documents/PCR/Bangalore_Prices.xlsx")

In [574]:
df1.columns

Index(['custcode', 'Custname', 'ProductVal', 'Invoiceno', 'Invcdate', 'InvVal',
       'Gpno', 'Qty', 'Rate', 'Itemcode', 'Descr', 'Phy_Attrib1',
       'Phy_Attrib2', 'Orderdate', 'Transportr', 'Pordno', 'Porddt', 'FrtTerm',
       'Market', 'Segment', 'Segment1', 'Segment2', 'ShortDesc',
       'Consignee Name', 'Dlv Term'],
      dtype='object')

In [575]:
df1 = df1.dropna(subset=['Market'])

# RFM Algorithm

In [576]:
df1['Invcdate'].min()

Timestamp('2017-01-04 00:00:00')

In [577]:
df1['Invcdate'].max()

Timestamp('2019-12-31 00:00:00')

In [578]:
import datetime as dt
NOW = dt.datetime(2019,12,31)
df1['Invcdate'] = pd.to_datetime(df1['Invcdate'])

In [579]:
rfm = df1.groupby('custcode', as_index=False).agg({'Invcdate': lambda x: (NOW - x.max()).days, 'Invoiceno': lambda x: len(x), 'InvVal': lambda x: x.sum()})
rfm['Invcdate'] = rfm['Invcdate'].astype(int)
rfm.rename(columns={'Invcdate': 'Recency', 
                         'Invoiceno': 'Frequency', 
                         'InvVal': 'Monetary'}, inplace=True)

In [580]:
quintiles = rfm[['Recency', 'Frequency', 'Monetary']].quantile([.2, .4, .6, .8]).to_dict()

In [581]:
def r_score(x):
    if x <= quintiles['Recency'][.2]:
        return 1
    elif x <= quintiles['Recency'][.4]:
        return 2
    elif x <= quintiles['Recency'][.6]:
        return 3
    elif x <= quintiles['Recency'][.8]:
        return 4
    else:
        return 5
    
def fm_score(x, c):
    if x <= quintiles[c][.2]:
        return 5
    elif x <= quintiles[c][.4]:
        return 4
    elif x <= quintiles[c][.6]:
        return 3
    elif x <= quintiles[c][.8]:
        return 2
    else:
        return 1  

In [582]:
rfm['R'] = rfm['Recency'].apply(lambda x: r_score(x))
rfm['F'] = rfm['Frequency'].apply(lambda x: fm_score(x, 'Frequency'))
rfm['M'] = rfm['Monetary'].apply(lambda x: fm_score(x, 'Monetary'))

In [583]:
rfm['RFM Score'] = rfm['R'].map(str) + rfm['F'].map(str) + rfm['M'].map(str)
rfm.head()

,custcode,Recency,Frequency,Monetary,R,F,M,RFM Score
0,930249,0,82,7711994.80,1,1,2,112
1,930261,826,13,1040300.44,5,2,4,524
2,930386,1020,1,2670.21,5,5,5,555
3,930588,181,15,2197941.00,3,2,3,323
4,930615,0,31,4917974.34,1,1,2,112


In [584]:
segt_map = {
    r'[1-2][1-2]': 'Champion',
    r'[1-2][3-4]': 'Loyal Customers',
    r'[1-2]5': 'Potential Loyalist',
    r'3[1-2]': 'Recent Customers',
    r'33': 'Promising',
    r'[3-4][4-5]': 'Customers Needing Attention',
    r'41': 'About to Sleep',
    r'51': 'At Risk',
    r'[4-5][2-3]': 'At Risk',
    r'5[4-5]': 'Hibernating'
}

rfm['Attribute'] = rfm['R'].map(str) + rfm['F'].map(str)
rfm['Attribute'] = rfm['Attribute'].replace(segt_map, regex=True)
rfm.head()

,custcode,Recency,Frequency,Monetary,R,F,M,RFM Score,Attribute
0,930249,0,82,7711994.80,1,1,2,112,Champion
1,930261,826,13,1040300.44,5,2,4,524,At Risk
2,930386,1020,1,2670.21,5,5,5,555,Hibernating
3,930588,181,15,2197941.00,3,2,3,323,Recent Customers
4,930615,0,31,4917974.34,1,1,2,112,Champion


In [585]:
tier_map = {
    r'[1-2][1-2][1-2]': 'Platinum',
    
    r'[1-2][3-4][1-2]': 'Gold',
    r'[3-4][1-2][1-2]': 'Gold',
    
    r'[1-2][1-2]3': 'Silver',
    r'[2-3][1-3][1-3]': 'Silver',
    r'4[1-2][1-3]': 'Silver',
    r'[1-2][3-4]3': 'Silver',
    r'[1-2][1-2][4-5]': 'Silver',
    r'[1-2][4-5][1-2]': 'Silver',
    r'[1-2][3-4][4-5]': 'Silver',
    r'[4-5][3-4][1-2]': 'Silver',
    r'[4-5][1-2][1-2]': 'Silver',
    
    
    
    r'[3-5][4-5][1-2]': 'Bronze',
    r'[3-5][3-5][3-5]': 'Bronze',
    r'[4-5][4-5][3-5]': 'Bronze',
    r'[4-5][1-2][4-5]': 'Bronze',
    r'[1-2][4-5][3-5]': 'Bronze',
    r'[3-4][1-2][4-5]': 'Bronze',
    
    
}

rfm['Tier'] = rfm['R'].map(str) + rfm['F'].map(str) + rfm['M'].map(str)
rfm['Tier'] = rfm['Tier'].replace(tier_map, regex=True)
rfm.head()

,custcode,Recency,Frequency,Monetary,R,F,M,RFM Score,Attribute,Tier
0,930249,0,82,7711994.80,1,1,2,112,Champion,Platinum
1,930261,826,13,1040300.44,5,2,4,524,At Risk,Bronze
2,930386,1020,1,2670.21,5,5,5,555,Hibernating,Bronze
3,930588,181,15,2197941.00,3,2,3,323,Recent Customers,Silver
4,930615,0,31,4917974.34,1,1,2,112,Champion,Platinum


In [586]:
rfm.Tier.unique()

array(['Platinum', 'Bronze', 'Silver', 'Gold'], dtype=object)

In [587]:
df1=df1.drop(['Phy_Attrib1', 'Phy_Attrib2','Segment1', 'Segment2', 
            'Gpno', 'ShortDesc', 'Consignee Name', 'Dlv Term','Transportr', 'Pordno','Porddt', 'FrtTerm'], axis=1)

In [588]:
rfm = rfm.drop_duplicates()

## Merge the labeled RFM dataframe to Original dataframe
df=pd.merge(df1, rfm , on=['custcode'])

In [589]:
df_lvl2=df.groupby(['Tier','Itemcode']).agg({'Rate':'mean'}).reset_index()
df_lvl2=df_lvl2.rename(columns={'Rate': 'Mean_Rate'})
df2=pd.merge(df, df_lvl2, on=['Tier', 'Itemcode'])
df2['Leakage']=df2['Rate']-df2['Mean_Rate']
df2['Leakage_perc']=(df2['Leakage']/df2['Rate'])*100
df2.head()

,custcode,Custname,ProductVal,Invoiceno,Invcdate,InvVal,Qty,Rate,Itemcode,Descr,...,Monetary,R,F,M,RFM Score,Attribute,Tier,Mean_Rate,Leakage,Leakage_perc
0,930936,ZIRCAST LTD,3875.0,BDI9703376,2017-01-27,4598.75,25,155.0,PA 748,REFRACTORY PATCHING MATERIAL PA-748,...,202388.68,4,3,5,435,At Risk,Bronze,150.790426,4.209574,2.715854
1,930936,ZIRCAST LTD,7750.0,BDI9706747,2017-02-20,9198.55,50,155.0,PA 748,REFRACTORY PATCHING MATERIAL PA-748,...,202388.68,4,3,5,435,At Risk,Bronze,150.790426,4.209574,2.715854
2,930936,ZIRCAST LTD,17000.0,BDI9716153,2017-04-20,20176.88,100,170.0,PA 748,REFRACTORY PATCHING MATERIAL PA-748,...,202388.68,4,3,5,435,At Risk,Bronze,150.790426,19.209574,11.299750
3,930936,ZIRCAST LTD,34000.0,BDI9728394,2017-07-10,40120.00,200,170.0,PA 748,REFRACTORY PATCHING MATERIAL PA-748,...,202388.68,4,3,5,435,At Risk,Bronze,150.790426,19.209574,11.299750
4,930936,ZIRCAST LTD,18190.0,BDI9801557,2018-01-08,21464.20,100,181.9,PA 748,REFRACTORY PATCHING MATERIAL PA-748,...,202388.68,4,3,5,435,At Risk,Bronze,150.790426,31.109574,17.102570


In [590]:
df2['Orderdate'] = pd.to_datetime(df2['Orderdate'])
df2['Duration']=(df2.Invcdate-df2.Orderdate).astype('timedelta64[h]')
df_dur = df2.groupby('custcode', as_index=False).agg({'Duration': [min, max, 'mean']})
df_dur.columns = ["_".join(x) for x in df_dur.columns.ravel()]

In [591]:
df_dur=df_dur.rename(columns={'custcode_': 'custcode'})

In [592]:
result_df=pd.merge(df_dur , df2 , on=['custcode'])

In [593]:
result_df.to_csv(r'C:/Users/L7927301/Documents/PCR/Total_df.csv')

In [594]:
df=df.drop(['Descr', 'Recency', 'Frequency','Monetary', 'R','F','M','RFM Score','Attribute','Tier'], axis=1)

In [595]:
mask_2017 = (df['Invcdate'] >= '2017-01-01') & (df['Invcdate'] < '2018-01-01')
mask_2018 = (df['Invcdate'] >= '2018-01-01') & (df['Invcdate'] < '2019-01-01')
mask_2019 = (df['Invcdate'] >= '2019-01-01') & (df['Invcdate'] < '2020-01-01')

In [596]:
df_2017=df.loc[mask_2017]
df_2018=df.loc[mask_2018]
df_2019=df.loc[mask_2019]

In [597]:
NOW_2017 = dt.datetime(2017,12,31)
df_2017['Invcdate'] = pd.to_datetime(df_2017['Invcdate'])

NOW_2018 = dt.datetime(2018,12,31)
df_2018['Invcdate'] = pd.to_datetime(df_2018['Invcdate'])

NOW_2019 = dt.datetime(2019,12,31)
df_2019['Invcdate'] = pd.to_datetime(df_2019['Invcdate'])

C:\Users\L7927301\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\L7927301\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
C:\Users\L7927301\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] 

In [598]:
rfm1 = df_2017.groupby('custcode', as_index=False).agg({'Invcdate': lambda x: (NOW_2017 - x.max()).days, 'Invoiceno': lambda x: len(x), 'InvVal': lambda x: x.sum()})
rfm1['Invcdate'] = rfm1['Invcdate'].astype(int)
rfm1.rename(columns={'Invcdate': 'Recency', 
                         'Invoiceno': 'Frequency', 
                         'InvVal': 'Monetary'}, inplace=True)

In [599]:
rfm2 = df_2018.groupby('custcode', as_index=False).agg({'Invcdate': lambda x: (NOW_2018 - x.max()).days, 'Invoiceno': lambda x: len(x), 'InvVal': lambda x: x.sum()})
rfm2['Invcdate'] = rfm2['Invcdate'].astype(int)
rfm2.rename(columns={'Invcdate': 'Recency', 
                         'Invoiceno': 'Frequency', 
                         'InvVal': 'Monetary'}, inplace=True)

In [600]:
rfm3 = df_2019.groupby('custcode', as_index=False).agg({'Invcdate': lambda x: (NOW_2019 - x.max()).days, 'Invoiceno': lambda x: len(x), 'InvVal': lambda x: x.sum()})
rfm3['Invcdate'] = rfm3['Invcdate'].astype(int)
rfm3.rename(columns={'Invcdate': 'Recency', 
                         'Invoiceno': 'Frequency', 
                         'InvVal': 'Monetary'}, inplace=True)

In [601]:
rfm1['R'] = rfm1['Recency'].apply(lambda x: r_score(x))
rfm1['F'] = rfm1['Frequency'].apply(lambda x: fm_score(x, 'Frequency'))
rfm1['M'] = rfm1['Monetary'].apply(lambda x: fm_score(x, 'Monetary'))

rfm2['R'] = rfm2['Recency'].apply(lambda x: r_score(x))
rfm2['F'] = rfm2['Frequency'].apply(lambda x: fm_score(x, 'Frequency'))
rfm2['M'] = rfm2['Monetary'].apply(lambda x: fm_score(x, 'Monetary'))

rfm3['R'] = rfm3['Recency'].apply(lambda x: r_score(x))
rfm3['F'] = rfm3['Frequency'].apply(lambda x: fm_score(x, 'Frequency'))
rfm3['M'] = rfm3['Monetary'].apply(lambda x: fm_score(x, 'Monetary'))

In [602]:
rfm1['RFM Score'] = rfm1['R'].map(str) + rfm1['F'].map(str) + rfm1['M'].map(str)
rfm2['RFM Score'] = rfm2['R'].map(str) + rfm2['F'].map(str) + rfm2['M'].map(str)
rfm3['RFM Score'] = rfm3['R'].map(str) + rfm3['F'].map(str) + rfm3['M'].map(str)

In [603]:
rfm1['Attribute'] = rfm1['R'].map(str) + rfm1['F'].map(str)
rfm1['Attribute'] = rfm1['Attribute'].replace(segt_map, regex=True)

rfm2['Attribute'] = rfm2['R'].map(str) + rfm2['F'].map(str)
rfm2['Attribute'] = rfm2['Attribute'].replace(segt_map, regex=True)

rfm3['Attribute'] = rfm3['R'].map(str) + rfm3['F'].map(str)
rfm3['Attribute'] = rfm3['Attribute'].replace(segt_map, regex=True)



rfm1['Tier'] = rfm1['R'].map(str) + rfm1['F'].map(str) + rfm1['M'].map(str)
rfm1['Tier'] = rfm1['Tier'].replace(tier_map, regex=True)

rfm2['Tier'] = rfm2['R'].map(str) + rfm2['F'].map(str) + rfm2['M'].map(str)
rfm2['Tier'] = rfm2['Tier'].replace(tier_map, regex=True)

rfm3['Tier'] = rfm3['R'].map(str) + rfm3['F'].map(str) + rfm3['M'].map(str)
rfm3['Tier'] = rfm3['Tier'].replace(tier_map, regex=True)

In [604]:
rfm3.Tier.unique()

array(['Silver', 'Bronze', 'Gold', 'Platinum'], dtype=object)

In [605]:
rfm1 = rfm1.drop_duplicates()
rfm2 = rfm2.drop_duplicates()
rfm3 = rfm3.drop_duplicates()

## Merge the labeled RFM dataframe to Original dataframe
df_17=pd.merge(df_2017, rfm , on=['custcode'])
df_18=pd.merge(df_2018, rfm , on=['custcode'])
df_19=pd.merge(df_2019, rfm , on=['custcode'])


In [606]:

df_17.to_csv(r'C:/Users/L7927301/Documents/PCR/df_2017.csv')
df_18.to_csv(r'C:/Users/L7927301/Documents/PCR/df_2018.csv')
df_19.to_csv(r'C:/Users/L7927301/Documents/PCR/df_2019.csv')

In [607]:
df.columns

Index(['custcode', 'Custname', 'ProductVal', 'Invoiceno', 'Invcdate', 'InvVal',
       'Qty', 'Rate', 'Itemcode', 'Orderdate', 'Market', 'Segment'],
      dtype='object')

In [608]:
df_19.columns

Index(['custcode', 'Custname', 'ProductVal', 'Invoiceno', 'Invcdate', 'InvVal',
       'Qty', 'Rate', 'Itemcode', 'Orderdate', 'Market', 'Segment', 'Recency',
       'Frequency', 'Monetary', 'R', 'F', 'M', 'RFM Score', 'Attribute',
       'Tier'],
      dtype='object')

In [609]:
df_a = df_17.append(df_18) 
df_merged = df_a.append(df_19)

In [610]:
df_merged.to_csv(r'C:/Users/L7927301/Documents/PCR/df_merged.csv')